In [1]:
from models.classifier import TransformerEncoder
from pytorch_lightning import Trainer
import torch

In [2]:
hparams = {
    "hidden_size": 48,  # size of the hidden layers and embeddings
    "hidden_ff": 96,  # size of the position-wise feed-forward layer
    "n_encoders": 4,  # number of encoder blocks
    "n_heads": 2,  # number of attention heads in the multiheadattention module
    "n_local": 2,  # number of local attention heads
    "local_window_size": 4,  # size of the window for local attention
    'batch_size': 4,
    "max_length": 30,  # maximum length of the input sequence
    "vocab_size": 100,  # size of the vocabulary
    "learning_rate": 0.001,
    "num_epochs": 30,
    "attention_type": "performer",
    "norm_type": "rezero",
    "num_random_features": 32,  # number of random features for the Attention module (Performer uses this)
    "emb_dropout": 0.1,  # dropout for the embedding block
    "fw_dropout": 0.1,  # dropout for the position-wise feed-forward layer
    "att_dropout": 0.1,  # dropout for the multiheadattention module
    "dc_dropout": 0.1,  # dropout for the decoder block
    "hidden_act": "swish",  # activation function for the hidden layers (attention layers use ReLU)
    "epsilon": 1e-8,
    "weight_decay": 0.01,
    "beta1": 0.9,
    "beta2": 0.999,
}
model = TransformerEncoder(hparams)

#### Sanity Check

In [3]:
batch = {
    'tokens': torch.randint(0, model.hparams.vocab_size, (model.hparams.batch_size, model.hparams.max_length)),
    'abspos': torch.arange(0, model.hparams.max_length).unsqueeze(0).repeat(model.hparams.batch_size, 1),
    'age': torch.randint(0, 100, (model.hparams.batch_size, 1)),
    'padding_mask': torch.zeros(model.hparams.batch_size, model.hparams.max_length),
    'targets': torch.randint(0, 2, (model.hparams.batch_size, 1)).float(),
}

In [4]:
model(batch)

{'logits': tensor([[-0.2905],
         [-0.0947],
         [-0.3562],
         [-0.0159]], grad_fn=<AddmmBackward0>),
 'preds': tensor([[0.4279],
         [0.4764],
         [0.4119],
         [0.4960]], grad_fn=<SigmoidBackward0>)}

In [5]:
model.training_step(batch, 0)

tensor(0.7724, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)

In [6]:
model.on_train_epoch_end()


Train Metrics
	Loss: 0.772
	Accuracy: 0.500
	MCC: 0.000



/Users/carlomarx/.local/share/virtualenvs/life2vec-light-Ez8u7ZRp-python/lib/python3.11/site-packages/pytorch_lightning/core/module.py:447: You are trying to `self.log()` but the `self.trainer` reference is not registered on the model yet. This is most likely because the model hasn't been passed to the `Trainer`


### Full pipeline would be something like

In [7]:
from dataloaders.synthetic import SyntheticDataModule
dataloader = SyntheticDataModule(num_samples=1000, max_length=hparams['max_length'],
                                  batch_size=hparams['batch_size'], vocab_size=hparams['vocab_size'])

In [ ]:
trainer = Trainer(max_epochs=30,
                accelerator="cpu",   ### change to "cuda" or "gpu" or 'msp'
                limit_train_batches=0.5,
                accumulate_grad_batches=4,
                num_sanity_val_steps=8,
                check_val_every_n_epoch=1)
trainer.fit(model, dataloader)

GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/Users/carlomarx/.local/share/virtualenvs/life2vec-light-Ez8u7ZRp-python/lib/python3.11/site-packages/pytorch_lightning/trainer/setup.py:177: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`.

   | Name        | Type                   | Params | Mode
---------------------------------------------------------------
0  | transformer | Transformer            | 79.5 K | eval
1  | decoder     | CLS_Decoder            | 2.4 K  | eval
2  | loss        | BCEWithLogitsLoss      | 0      | eval
3  | train_loss  | MeanMetric             | 0      | eval
4  | valid_loss  | MeanMetric             | 0      | eval
5  | test_loss   | MeanMetric             | 0      | eval
6  | train_acc   | BinaryAccuracy         | 0      | eval
7  | valid_acc   | BinaryAccuracy         | 0      | eval
8  | test_acc    | BinaryAccuracy         | 0      | eval
9  | train_mcc 

/Users/carlomarx/.local/share/virtualenvs/life2vec-light-Ez8u7ZRp-python/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:424: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=9` in the `DataLoader` to improve performance.
/Users/carlomarx/.local/share/virtualenvs/life2vec-light-Ez8u7ZRp-python/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:424: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=9` in the `DataLoader` to improve performance.


Epoch 0: 100%|██████████| 125/125 [00:03<00:00, 35.50it/s, v_num=10]
Train Metrics
	Loss: 0.695
	Accuracy: 0.492
	MCC: -0.016

Epoch 1: 100%|██████████| 125/125 [00:03<00:00, 39.28it/s, v_num=10]
Train Metrics
	Loss: 0.694
	Accuracy: 0.494
	MCC: -0.015

Epoch 2: 100%|██████████| 125/125 [00:03<00:00, 38.55it/s, v_num=10]
Train Metrics
	Loss: 0.694
	Accuracy: 0.497
	MCC: -0.012

Epoch 3: 100%|██████████| 125/125 [00:03<00:00, 39.15it/s, v_num=10]
Train Metrics
	Loss: 0.694
	Accuracy: 0.502
	MCC: -0.007

Epoch 4: 100%|██████████| 125/125 [00:03<00:00, 37.06it/s, v_num=10]
Train Metrics
	Loss: 0.694
	Accuracy: 0.499
	MCC: -0.011

Epoch 5:  70%|██████▉   | 87/125 [00:01<00:00, 77.50it/s, v_num=10] 

/Users/carlomarx/.local/share/virtualenvs/life2vec-light-Ez8u7ZRp-python/lib/python3.11/site-packages/pytorch_lightning/trainer/call.py:54: Detected KeyboardInterrupt, attempting graceful shutdown...
